# Tutorial 3: Loading and inspecting ME-models

In this tutorial we will load and inspect the reconstructed _Bacillus subtilis_ ME-model.

## Import libraries

In [1]:
from coralme.builder.main import MEBuilder
import coralme

## Load

Load the ME-model coming out of the Troubleshooter

In [2]:
me = coralme.io.json.load_json_me_model("MEModel-step3-bsubtilis-TS.json")

Set parameter Username
Academic license - for non-commercial use only - expires 2024-08-16


Adding Metabolites into the ME-model...                                    : 100.0%|██████████|  4432/ 4432 [00:00<00:00]
Adding ProcessData into the ME-model...                                    : 100.0%|██████████|  4484/ 4484 [00:00<00:00]
Adding Reactions into the ME-model...                                      : 100.0%|██████████|  7461/ 7461 [00:16<00:00]
Updating ME-model Reactions...                                             : 100.0%|██████████|  6068/ 6068 [00:17<00:00]


## Solve

In [3]:
me.optimize()

The MINOS and quad MINOS solvers are a courtesy of Prof Michael A. Saunders. Please cite Ma, D., Yang, L., Fleming, R. et al. Reliable and efficient solution of genome-scale models of Metabolism and macromolecular Expression. Sci Rep 7, 40863 (2017). https://doi.org/10.1038/srep40863

Iteration	 Solution to check	Solver Status
---------	------------------	-------------
        1	1.4050280687025918	Not feasible
        2	0.7025140343512959	Not feasible
        3	0.3512570171756479	Not feasible
        4	0.1756285085878240	Not feasible
        5	0.0878142542939120	Optimal
        6	0.1317213814408680	Not feasible
        7	0.1097678178673900	Not feasible
        8	0.0987910360806510	Not feasible
        9	0.0933026451872815	Not feasible
       10	0.0905584497405967	Optimal
       11	0.0919305474639391	Not feasible
       12	0.0912444986022679	Optimal
       13	0.0915875230331035	Optimal
       14	0.0917590352485213	Optimal
       15	0.0918447913562302	Optimal
       16	0.0918876694100847

True

## Inspecting ME-model properties

### Summary of the ME-model

In [4]:
me

Name,coralME
Memory address,0x07f733a203df0
Number of metabolites,4432
Number of reactions,7461
Number of groups,0
Objective expression,0
Compartments,"mc, c, e, im, p"


In [5]:
print("ME-model has {} reactions".format(len(me.reactions)))

ME-model has 7461 reactions


In [6]:
print("ME-model has {} metabolites".format(len(me.metabolites)))

ME-model has 4432 metabolites


In [7]:
print("ME-model has {} genes".format(len(me.all_genes)))

ME-model has 1103 genes


### Reactions

#### Metabolic reaction

In [8]:
r = next(i for i in me.reactions if isinstance(i,coralme.core.reaction.MetabolicReaction))
r

Reaction identifier,23CN2P1_REV_BSU07840-MONOMER
Name,
Memory address,0x7f71c85dde40
Stoichiometry,"1.0 3amp_c + [1.23635177537696e-6*mu1] BSU07840-MONOMER + 1.0 h_c --> 1.0 23camp_c + 1.0 h2o_c 1.0 3 AMP C10H12N5O7P + [1.23635177537696e-6*mu1] BSU07840-MONOMER + 1.0 H+ --> 1.0 2',3'-Cyclic AMP + 1.0 H2O H2O"
GPR,
Lower bound,0
Upper bound,999999.0
Reaction type,coralme.core.reaction.MetabolicReaction
Flux,0 ($\mu$= 0.0919084)
Reduced cost,-4.6591e-07 ($\mu$= 0.0919084)
Bound violation,False


#### Translation reaction

In [9]:
r = next(i for i in me.reactions if isinstance(i,coralme.core.reaction.TranslationReaction))
r

Reaction identifier,translation_BSU00090
Name,
Memory address,0x7f71c98f7cd0
Stoichiometry,1.0 10fthf_c + [0.00208119658119658*mu1] BSU01120-MONOMER + [4.27350427350427e-6*mu1] BSU01390-MONOMER + [4.27350427350427e-6*mu1] BSU15720-MONOMER_mod_fe2(1) + [4.27350427350427e-6*mu1] BSU15730-MONOMER_mod_mg2(1)_mod_k(1) + [0.00208119658119658*mu1] BSU16500-MONOMER + [4.27350427350427e-6*mu1] BSU16520-MONOMER + [8.54700854700855e-6*mu1] BSU16630-MONOMER + [4.27350427350427e-6*mu1] BSU25460-MONOMER_mod_zn2(4) + [4.27350427350427e-6*mu1] BSU25470-MONOMER + [4.27350427350427e-6*mu1] BSU25480-MONOMER + [4.27350427350427e-6*mu1] BSU28870-MONOMER + [0.000498399634202103*mu1 + 0.000195123456790123 + 0.00598079561042524*(mu1 + 0.3915)/mu1] RNA_BSU00090 + [2.55589556001079e-8*mu1 + 1.00063311174422e-8] RNA_degradosome + 36 ala__L_c + 23 arg__L_c + 14 asn__L_c + 27 asp__L_c + [488.0 + 2.19196159122085*(mu1 + 0.3915)/mu1] atp_c + 3 cys__L_c + [4.27350427350427e-6*mu1] generic_RF + [0.00208119658119658*mu1] generic_Tuf + 29.0 generic_tRNA_AAA_lys__L_c + 8.0 generic_tRNA_AAC_asn__L_c + 8.0... 1.0 10-Formyltetrahydrofolate + [0.00208119658119658*mu1] BSU01120-MONOMER + [4.27350427350427e-6*mu1] BSU01390-MONOMER + [4.27350427350427e-6*mu1] BSU15720-MONOMER_mod_fe2(1) + [4.27350427350427e-6*mu1] BSU15730-MONOMER_mod_mg2(1)_mod_k(1) + [0.00208119658119658*mu1] BSU16500-MONOMER + [4.27350427350427e-6*mu1] BSU16520-MONOMER + [8.54700854700855e-6*mu1] BSU16630-MONOMER + [4.27350427350427e-6*mu1] BSU25460-MONOMER_mod_zn2(4) + [4.27350427350427e-6*mu1] BSU25470-MONOMER + [4.27350427350427e-6*mu1] BSU25480-MONOMER + [4.27350427350427e-6*mu1] BSU28870-MONOMER + [0.000498399634202103*mu1 + 0.000195123456790123 + 0.00598079561042524*(mu1 + 0.3915)/mu1] RNA_BSU00090 + [2.55589556001079e-8*mu1 + 1.00063311174422e-8] RNA_degradosome + 36 L-Alanine + 23 L-Arginine + 14 L-Asparagine + 27 L-Aspartate + [488.0 + 2.19196159122085*(mu1 + 0.3915)/mu1] ATP C10H12N5O13P3 + 3 L-Cysteine + [4.27350427350427e-6*mu1] generic_RF + [0.00208119658119658*mu1] generic_Tuf + 29.0 generic_tRNA_AAA_lys__L_c...
GPR,
Lower bound,0.0
Upper bound,1000.0
Reaction type,coralme.core.reaction.TranslationReaction
Flux,3.92632e-09 ($\mu$= 0.0919084)
Reduced cost,-1.85222e-32 ($\mu$= 0.0919084)
Bound violation,False


#### Transcription reaction

In [10]:
r = next(i for i in me.reactions if isinstance(i,coralme.core.reaction.TranscriptionReaction))
r

Reaction identifier,transcription_TU8J2-1223_from_BSU25200-MONOMER
Name,
Memory address,0x7f71ca852d10
Stoichiometry,[4.27350427350427e-6*mu1] BSU00550-MONOMER_mod_mg2(1) + [4.27350427350427e-6*mu1] BSU01010-MONOMER + [4.27350427350427e-6*mu1] BSU16600-MONOMER + [4.27350427350427e-6*mu1] BSU27320-MONOMER + [4.27350427350427e-6*mu1] BSU37080-MONOMER_mod_mg2(3) + [0.00736902347633249*mu1 + 0.00288497269098417] RNAP_BSU25200MONOMER + 369.0 atp_c + 432 ctp_c + 330 gtp_c + 3.0 h2o_c + 414 utp_c --> 1.0 RNA_BSU10270 + 3.0 adp_c + 3.0 h_c + 491.443913982 mRNA_biomass + 3.0 pi_c + 1542 ppi_c [4.27350427350427e-6*mu1] BSU00550-MONOMER_mod_mg2(1) + [4.27350427350427e-6*mu1] BSU01010-MONOMER + [4.27350427350427e-6*mu1] BSU16600-MONOMER + [4.27350427350427e-6*mu1] BSU27320-MONOMER + [4.27350427350427e-6*mu1] BSU37080-MONOMER_mod_mg2(3) + [0.00736902347633249*mu1 + 0.00288497269098417] RNAP_BSU25200MONOMER + 369.0 ATP C10H12N5O13P3 + 432 CTP C9H12N3O14P3 + 330 GTP C10H12N5O14P3 + 3.0 H2O H2O + 414 UTP C9H11N2O15P3 --> 1.0 RNA_BSU10270 + 3.0 ADP C10H12N5O10P2 + 3.0 H+ + 491.443913982 mRNA_biomass + 3.0 Phosphate + 1542 Diphosphate
GPR,
Lower bound,0.0
Upper bound,1000.0
Reaction type,coralme.core.reaction.TranscriptionReaction
Flux,0 ($\mu$= 0.0919084)
Reduced cost,2.139e-32 ($\mu$= 0.0919084)
Bound violation,False


#### Formation reaction

In [11]:
r = next(i for i in me.reactions if isinstance(i,coralme.core.reaction.ComplexFormation))
r

Reaction identifier,formation_BSU00090-MONOMER
Name,
Memory address,0x7f71c84544c0
Stoichiometry,4.0 protein_BSU00090 --> 1.0 BSU00090-MONOMER 4.0 protein_BSU00090 --> 1.0 BSU00090-MONOMER
GPR,
Lower bound,0.0
Upper bound,1000.0
Reaction type,coralme.core.reaction.ComplexFormation
Flux,9.8158e-10 ($\mu$= 0.0919084)
Reduced cost,0 ($\mu$= 0.0919084)
Bound violation,False


#### tRNA Charging reaction

In [12]:
r = next(i for i in me.reactions if isinstance(i,coralme.core.reaction.tRNAChargingReaction))
r

Reaction identifier,charging_tRNA_BSU_tRNA_5_AUU
Name,
Memory address,0x7f71c85fe7a0
Stoichiometry,[4.27350427350427e-6*mu1*(0.000116266666666667*mu1 + 1.0000455184)] BSU15430-MONOMER + [0.000116266666666667*mu1 + 4.55184e-5] RNA_BSU_tRNA_5 + [0.000116266666666667*mu1 + 4.55184e-5] ile__L_c --> 1.0 generic_tRNA_AUU_ile__L_c [4.27350427350427e-6*mu1*(0.000116266666666667*mu1 + 1.0000455184)] BSU15430-MONOMER + [0.000116266666666667*mu1 + 4.55184e-5] RNA_BSU_tRNA_5 + [0.000116266666666667*mu1 + 4.55184e-5] L-Isoleucine --> 1.0 generic_tRNA_AUU_ile__L_c
GPR,
Lower bound,0.0
Upper bound,1000.0
Reaction type,coralme.core.reaction.tRNAChargingReaction
Flux,0.0288233 ($\mu$= 0.0919084)
Reduced cost,-5.48188e-36 ($\mu$= 0.0919084)
Bound violation,False


#### Post-translational modification reaction

In [13]:
r = next(i for i in me.reactions if isinstance(i,coralme.core.reaction.PostTranslationReaction))
r

Reaction identifier,translocation_BSU07840_Cell_Wall
Name,
Memory address,0x7f71c3b064a0
Stoichiometry,[0.101527777777778*mu1] BSU35300-MONOMER + 58.480000000000004 atp_c + 58.480000000000004 h2o_c + 1.0 protein_BSU07840 --> 58.480000000000004 adp_c + 58.480000000000004 h_c + 58.480000000000004 pi_c + 1.0 protein_BSU07840_Cell_Wall [0.101527777777778*mu1] BSU35300-MONOMER + 58.480000000000004 ATP C10H12N5O13P3 + 58.480000000000004 H2O H2O + 1.0 protein_BSU07840 --> 58.480000000000004 ADP C10H12N5O10P2 + 58.480000000000004 H+ + 58.480000000000004 Phosphate + 1.0 protein_BSU07840_Cell_Wall
GPR,
Lower bound,0.0
Upper bound,1000.0
Reaction type,coralme.core.reaction.PostTranslationReaction
Flux,9.34868e-12 ($\mu$= 0.0919084)
Reduced cost,-1.14226e-33 ($\mu$= 0.0919084)
Bound violation,False


### Mapped gene functions

In [14]:
from collections import defaultdict
import pandas
d = defaultdict(int)
for g in me.all_genes:
    for f in g.functions:
        d[f] += 1
pandas.DataFrame.from_dict({"count":d})

,count
Metabolic:S_Amino_acids_and_related_molecules,155
Metabolic:S_Carbohydrates_and_related_molecules,156
Metabolic:S_Cell_wall,40
Metabolic:S_Coenzymes_and_prosthetic_groups,77
Metabolic:S_Lipids,55
Metabolic:S_Membrane_bioenergetics,47
Metabolic:S_Nucleotides_and_nucleic_acids,69
Metabolic:S_Other_function,15
Metabolic:S_Phosphate_and_sulfur,7
Metabolic:S_Transport,232


#### Inspecting functions of a gene

In [15]:
g = me.all_genes.get_by_id("RNA_BSU37160")
g.functions

{'Transcription'}

### Complexes formed by a gene

In [16]:
g = me.all_genes.get_by_id("RNA_BSU37160")
g.complexes

{<Complex CPLX8J2-30_mod_zn2(1)_mod_mg2(2) at 0x7f71caa3f490>,
 <RNAP RNAP_BSU25200MONOMER at 0x7f71caa3fa60>}

## Inspecting fluxes

### Predicted fluxes

In [17]:
fluxes = me.solution.to_frame()
fluxes

,fluxes,reduced_costs
biomass_dilution,9.190844e-02,-5.783041e-01
BSU00360-MONOMER_to_generic_16Sm4Cm1402,1.900832e-12,1.281207e-34
BSU15140-MONOMER_to_generic_16Sm4Cm1402,0.000000e+00,-1.071094e+01
RNA_BSU_rRNA_1_to_generic_16s_rRNAs,2.590240e-06,-1.904183e-32
RNA_BSU_rRNA_16_to_generic_16s_rRNAs,0.000000e+00,-6.431574e-02
...,...,...
TS_pydx5p_c,-4.360925e-07,0.000000e+00
TS_zn2_c,-2.351045e-07,0.000000e+00
TS_fe2_c,-1.273605e-09,0.000000e+00
TS_thmpp_c,-2.767482e-07,0.000000e+00


### Biomass production

In [18]:
fluxes[fluxes.index.str.contains("biomass")]

,fluxes,reduced_costs
biomass_dilution,0.091908,-5.783041e-01
protein_biomass_to_biomass,0.050029,-1.162165e-34
mRNA_biomass_to_biomass,0.000161,0.000000e+00
tRNA_biomass_to_biomass,0.000975,-6.057787e-35
rRNA_biomass_to_biomass,0.007090,0.000000e+00
ncRNA_biomass_to_biomass,0.000000,0.000000e+00
tmRNA_biomass_to_biomass,0.000000,0.000000e+00
DNA_biomass_to_biomass,0.005449,0.000000e+00
lipid_biomass_to_biomass,0.000000,0.000000e+00
constituent_biomass_to_biomass,0.000000,0.000000e+00


### Transcription rates

In [19]:
fluxes[fluxes.index.str.contains("^transcription_")].head()

,fluxes,reduced_costs
transcription_TU8J2-1223_from_BSU25200-MONOMER,0.000000e+00,2.138997e-32
transcription_TU8J2-1268_from_BSU25200-MONOMER,2.189610e-10,1.472155e-34
transcription_TU8J2-805_from_BSU25200-MONOMER,0.000000e+00,-1.025989e-31
transcription_TU8J2-1022_from_BSU25200-MONOMER,0.000000e+00,-4.382381e+02
transcription_TU8J2-458_from_BSU25200-MONOMER,2.542118e-08,-2.457499e-32


### Translation rates

In [20]:
fluxes[fluxes.index.str.contains("^translation_")].head()

,fluxes,reduced_costs
translation_BSU00090,3.926319e-09,-1.852217e-32
translation_BSU00110,0.000000e+00,8.759602e-34
translation_BSU00120,0.000000e+00,1.105340e-32
translation_BSU00130,3.028489e-08,9.921502e-32
translation_BSU00140,0.000000e+00,-5.574761e-34


### Sinks to be curated through manual curation

The Troubleshooter found 8 gaps that need to be curated to allow for growth. Most of these are cofactors that will require some synthesis pathways (e.g. 4fe4s), but others will only need some transporters (e.g. metal ions)

In [21]:
fluxes[fluxes.index.str.contains("^TS_")]

,fluxes,reduced_costs
TS_cbl1_c,-3.571466e-09,0.0
TS_cobalt2_c,-3.485413e-08,0.0
TS_mn2_c,-7.639833e-06,0.0
TS_pydx5p_c,-4.360925e-07,0.0
TS_zn2_c,-2.351045e-07,0.0
TS_fe2_c,-1.273605e-09,0.0
TS_thmpp_c,-2.767482e-07,0.0
TS_4fe4s_c,-2.335281e-07,0.0
